In [ ]:
import collections

class OrderedSetDict(collections.OrderedDict):
    def add(self, v):
        self[v] = v

    def remove(self, v):
        return self.pop(v)

# TODO NOTE data format {<callable>: <callable>} {<key>: <callable>}
class CallableSet(OrderedSetDict):
    def __call__(self, *args, **kwargs):
        return {
            f: f.__call__(*args, **kwargs)
                for _, f in self.items()
        }

In [ ]:
import abc
import typing

# TODO ref TimerBase
class BaseEventSource(abc.ABC):
    def __init__(self):
        self.callbacks = OrderedSetDict()

    def start(self):
        ...

    def stop(self):
        ...

    def add_callback(self, func: typing.Callable, *args, **kwargs):
        self.callbacks.add((func, args, kwargs))
        return func

    def remove_callback(self, func: typing.Callable, *args, **kwargs)
        self.callbacks.remove((func, args, kwargs))
        
    def step(self):
        for func, args, kwargs in self.callbacks:
            ret = func(*args, **kwargs)
            if ret == 0 or ret is False:
                self.callbacks.remove((func, args, kwargs))

In [ ]:
import matplotlib.animation

class BaseAnimation(matplotlib.animation.Animation):
    def __init__(
        self, 
        *args, 
        event_source: BaseEventSource = None, 
        **kwargs
    ):
        super().__init__(
            *args, 
            event_source=event_source or BaseEventSource(),
            **kwargs
        )

class ArtistAnimation(BaseAnimation):
    """
    `TimedAnimation` subclass that creates an animation by using a fixed
    set of `.Artist` objects.

    Before creating an instance, all plotting should have taken place
    and the relevant artists saved.

    .. note::

        You must store the created Animation in a variable that lives as long
        as the animation should run. Otherwise, the Animation object will be
        garbage-collected and the animation stops.

    Parameters
    ----------
    fig : `~matplotlib.figure.Figure`
        The figure object used to get needed events, such as draw or resize.
    artists : list
        Each list entry is a collection of `.Artist` objects that are made
        visible on the corresponding frame.  Other artists are made invisible.
    blit : bool, default: False
        Whether blitting is used to optimize drawing.
    """

    def __init__(self, fig, artists, *args, **kwargs):
        # Internal list of artists drawn in the most recent frame.
        self._drawn_artists = []

        # Use the list of artists as the framedata, which will be iterated
        # over by the machinery.
        self._framedata = artists
        super().__init__(fig, *args, **kwargs)

    def _init_draw(self):
        super()._init_draw()
        # Make all the artists involved in *any* frame invisible
        figs = set()
        for f in self.new_frame_seq():
            for artist in f:
                artist.set_visible(False)
                artist.set_animated(self._blit)
                # Assemble a list of unique figures that need flushing
                if artist.get_figure() not in figs:
                    figs.add(artist.get_figure())

        # Flush the needed figures
        for fig in figs:
            fig.canvas.draw_idle()

    def _pre_draw(self, framedata, blit):
        """Clears artists from the last frame."""
        if blit:
            # Let blit handle clearing
            self._blit_clear(self._drawn_artists)
        else:
            # Otherwise, make all the artists from the previous frame invisible
            for artist in self._drawn_artists:
                artist.set_visible(False)

    def _draw_frame(self, artists):
        # Save the artists that were passed in as framedata for the other
        # steps (esp. blitting) to use.
        self._drawn_artists = artists

        # Make all the artists from the current frame visible
        for artist in artists:
            artist.set_visible(True)

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

plt.ion()
fig, ax = plt.subplots()

# TODO
ArtistAnimation(
    fig.canvas, 
    [
        ax.add_artist(
            Line2D([], [])
            .set_factory(
                # TODO !!!! acc
                data=lambda: (
                    # TODO order!!!
                    (lambda xs: (range(len(xs)), xs))(data_agg())
                )
            )    
        )
    ]
)